This notebook computes the derivative of the resilience indicator. This information is usefull for rendering the scorcards. This notebook must be run after compute_res_ind and before render_scorecards

In [21]:
%reset-f
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from res_ind_lib import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# data

In [22]:
#Load the excel data file into a dataframe (table)
df_original=pd.read_excel("inputs/all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the first column (Provinces) as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variables and the 3rd (2) row is empty. We skip them
                ) 
df_original.index.name="province"
#Displays the first rows of that dataframe
df_original=def_ref_values(df_original)

# computes all derivatives

In [23]:
deriv_set = np.setdiff1d( df_original.columns,
    ["pop","iso3","original_country","income_elast","avg_prod_k","axfin_p","axfin_r","v_s","gdp_pc_pp","gdp_pc_pp_nat",
     "protectionref","gdp_pc_pp_ref","rho", "T_rebuild_K" ])

pd.DataFrame(data=deriv_set).to_csv("inputs/deriv_set.csv",index=False,header=False)

pol_assess_set = ["dKtot","dWtot_currency"]
              
#Checks that info has information for all variables in deriv_set
info = pd.read_csv("inputs/inputs_info.csv").set_index("key")
for d in deriv_set:
    if d not in info.index.values:
        raise Exception(d+" is not documented in inputs_info.csv")

        


In [24]:
from progress_reporter import *

###MH set adds and compares

def compute_derivative(df_original,deriv_set,pol_assess_set):
    der = pd.DataFrame(index=df_original.index, columns=pd.MultiIndex.from_product([deriv_set,pol_assess_set], names=['inputs', 'outputs']))
    h=0.0001
    #loop on all data in df prior to add the results
    fx = compute_resiliences(df_original)[pol_assess_set]
    for var in deriv_set:
        try:
            progress_reporter(var)
            df_=df_original.copy(deep=True)
            df_[var]=df_[var]+h
            fxh= compute_resiliences(df_)[pol_assess_set]
            der[var] = ((fxh-fx)/(h))  # for elasticity: .mul(df_[var],axis=0)/fx
        except TypeError:
            print("no derivative for " +var)
    progress_reporter("done.")        
    return der.swaplevel('inputs', 'outputs', axis=1).sort_index(axis=1).dropna(how="all",axis=1)

In [25]:
derivatives = compute_derivative(df_original,deriv_set,pol_assess_set)
derivatives.head()

Currently working on:  done.


outputs                     dKtot                                 \
inputs                        fap              far    nat_buyout   
province                                                           
Abra                227670.881587    919391.940185 -5.820766e-07   
Agusan Del Norte   1611098.429600   4131307.845861  2.328306e-06   
Agusan Del Sur     1995253.577513   2930588.077283  0.000000e+00   
Aklan              4894521.883042  14618905.100807  0.000000e+00   
Albay             16066028.839387  31889806.834252  0.000000e+00   

outputs                                                          \
inputs                        pi        pov_head     protection   
province                                                          
Abra              -212449.498333    55280.151948  -17061.320831   
Agusan Del Norte -1767215.686801  1325373.414119 -191108.275563   
Agusan Del Sur    -542593.069321   405790.353140  -64128.372923   
Aklan                   0.000000        0.000000       0.000000   
Albay                   0.000000        0.000000       0.000000   

outputs                                                                  \
inputs                rel_gdp_pp          share1           shew sigma_p   
province                                                                  
Abra               248393.163602    55219.470797  -44565.953396       0   
Agusan Del Norte  1849809.709750  1645235.240965 -430358.224248       0   
Agusan Del Sur     803777.188100   493566.611776 -132134.063514       0   
Aklan                   0.000000        0.000000       0.000000       0   
Albay                   0.000000        0.000000       0.000000       0   

outputs                ...       dWtot_currency                  \
inputs                 ...           protection      rel_gdp_pp   
province               ...                                        
Abra                   ...        -29498.709774  -214390.562393   
Agusan Del Norte       ...       -278929.462340 -1343243.251829   
Agusan Del Sur         ...       -134529.911757  -840068.101960   
Aklan                  ...             0.000000        0.000000   
Albay                  ...             0.000000        0.000000   

outputs                                                                       \
inputs                    share1           shew         sigma_p      sigma_r   
province                                                                       
Abra              -240138.657020  -77391.869536   -52502.979353  -490.688955   
Agusan Del Norte -3418827.164248 -636613.998073 -1587372.493092 -2548.001278   
Agusan Del Sur   -1127197.574561 -279702.737075  -636552.583866 -1086.132429   
Aklan                   0.000000       0.000000        0.000000     0.000000   
Albay                   0.000000       0.000000        0.000000     0.000000   

outputs                                                                        
inputs                  social_p     social_r             v_p             v_r  
province                                                                       
Abra               -59193.231873  -553.232633   654268.176447   452011.643804  
Agusan Del Norte -1729172.294829 -2775.614723  3985261.800843  2659576.228298  
Agusan Del Sur    -693415.744782 -1183.033106  2207240.581999   888572.223412  
Aklan                   0.000000     0.000000        0.000000        0.000000  
Albay                   0.000000     0.000000        0.000000        0.000000  

[5 rows x 30 columns]

# output

output signs in excel

In [26]:
#saves derivatives in excel tabs with signs in colors (very usefull for understanding the model)
writer= pd.ExcelWriter("results/derivatives.xlsx", engine='xlsxwriter')
workbook=writer.book
# Add a format. Light red fill with dark red text.
red = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

blue = workbook.add_format({'bg_color': '#92c5de',
                               'font_color': '#000061'})
for outname in pol_assess_set:
    sheetname = outname[0:30]
    (derivatives[outname].dropna()).to_excel(writer,sheet_name=sheetname)
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '>',
                                    'value':    0,
                                    'format':   blue})
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '<',
                                    'value':    0,
                                    'format':   red})
    writer.sheets[sheetname].freeze_panes(1, 1)


In [27]:
import warnings
warnings.filterwarnings("always",category=UserWarning)
try :
    writer.save()
except PermissionError:
    warnings.warn("Cannot write excel file. Check that it's not opened and try again")


# Double checks the results

In [28]:
#Signs of resilience derivative
for out in pol_assess_set:
    der =     np.sign(derivatives[out]).replace(0,np.nan)
    signs= pd.Series(index=der.columns)
    for i in signs.index:
        if (der[i].min()==der[i].max()): #all nonnan signs are equal
            signs[i]=der[i].min()
        else:
            print("ambigous sign for "+i+" on "+out)
            signs[i]=np.nan



ambigous sign for nat_buyout on dKtot
ambigous sign for sigma_p on dKtot
ambigous sign for sigma_r on dKtot
ambigous sign for social_p on dKtot
ambigous sign for social_r on dKtot
